# A notebook for exploring how best to measure the change in Mtb

In [1]:
import napari
from cellpose import models
from macrohet import dataio, tile, visualise, tools
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)

from napari_animation import Animation
from tqdm.auto import tqdm

import btrack
import dask.array as da

from skimage.transform import rescale, resize, downscale_local_mean

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("Spectral")


### Load all metadata

Both the image metadata and the assay layout metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Downscale function

In [106]:
def downscale_rgb_images(images, root_dir='/home/dayn/data/macrohet_temp/macrohet_images_ds/', 
                         segmentation=None, scale_factor=5.04):
    # iterate over frames
    for frame in tqdm(range(0, len(images)), desc='Iterating over frames', total=len(images)):
        
        # create output dir
        output_dir = os.path.join(root_dir, f'{row},{column}')
        os.makedirs(output_dir, exist_ok=True)
        # create RGB filename
        fn = os.path.join(output_dir, f'r0{row}c0{column}t{frame}.png')
        
        if os.path.exists(fn):
            continue
        
        # extract the gfp and rfp channels to apply some vis techn
        gfp = images[frame, 0, ...].compute().compute()
        rfp = images[frame, 1, ...].compute().compute()

        # clip the images so that the contrast is more apparent
        contrast_lim_gfp = np.clip(gfp, 358, 5886)
        contrast_lim_rfp = np.clip(rfp, 480, 1300)
        norm_gfp = cv2.normalize(contrast_lim_gfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        norm_rfp = cv2.normalize(contrast_lim_rfp, None, 0, 65535, cv2.NORM_MINMAX, dtype=cv2.CV_16U)

        # Create an empty RGB image with the same shape as the input image
        rgb_image = np.zeros((norm_gfp.shape[0], norm_gfp.shape[1], 3), dtype=np.uint16)

        # Assign the first channel to the green channel of the RGB image
        rgb_image[:, :, 1] = norm_gfp

        # Assign the second channel to the red and blue channels of the RGB image to create magenta
        rgb_image[:, :, 0] = norm_rfp
        rgb_image[:, :, 2] = norm_rfp

        # scale down to 8-bit
        rgb_image = np.uint8(rgb_image >> 8)

        # create mask outline
        if segmentation is not None:
            # get relevant frame
            masks = segmentation[frame]
            # iterate over each individual segment, drawing it onto the original RGB image
            for seg_ID in tqdm(range(1, np.max(masks)), desc='Creating outlines of masks', leave=False):
                instance_mask = (masks == seg_ID).astype(np.uint8)
                # draw outline
                contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cv2.drawContours(rgb_image, contours, -1, (0, 2**8, 2**8), thickness=2)  # make 8-bit

        # downsize image to reduce storage demands
        rgb_image = cv2.resize(rgb_image, (int(rgb_image.shape[1] // scale_factor), int(rgb_image.shape[0] // scale_factor)))

        # save out RGB image
        imsave(fn, rgb_image)

    print(f'Position {row}, {column} complete')


In [ ]:
for position, key in assay_layout.iterrows():
    
    if position == (3, 5):
        continue
        
    row = position[0]
    column = position[1]

    # pre-load images from set time point
    base_dir = '/mnt/DATA/macrohet/'
    image_dir = os.path.join(base_dir, 'macrohet_images/Images')
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, column, 
                                 set_plane='sum_proj',
                                 )
    ### load segmentation
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, 
                                                f'labels/macrohet_seg_model/{int(row),int(column)}.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
        segmentation = reader.segmentation
    
    # downscale
    downscale_rgb_images(images, 
                         root_dir = '/home/dayn/data/macrohet_temp/macrohet_images_ds/', 
                         segmentation = None, scale_factor=5.04)


# Multiprocess

In [ ]:
import os
import btrack.io
from functools import partial
import multiprocessing
import traceback

def process_position(position, base_dir, metadata):
    try:
        row = position[0]
        column = position[1]

        # pre-load images from set time point
        image_dir = os.path.join(base_dir, 'macrohet_images/Images')
        images = tile.compile_mosaic(image_dir,
                                     metadata,
                                     row, column,
                                     set_plane='sum_proj',
                                     )
        
        # load segmentation
        with btrack.io.HDF5FileHandler(os.path.join(base_dir,
                                                    f'labels/macrohet_seg_model/{int(row), int(column)}.h5'),
                                       'r',
                                       obj_type='obj_type_1') as reader:
            segmentation = reader.segmentation
        
        # downscale
        downscale_rgb_images(images,
                             root_dir='/home/dayn/data/macrohet_temp/macrohet_images_ds/',
                             segmentation=segmentation, scale_factor=5.04)
        
        print(f"Processing completed for position: {row}, {column}")
    except Exception as e:
        traceback.print_exc()
        print(f"Error occurred for position: {row}, {column}. Error: {str(e)}")

if __name__ == '__main__':
    # Assuming you have 'assay_layout' DataFrame with positions and keys
    
    base_dir = '/mnt/DATA/macrohet/'
    num_processes = multiprocessing.cpu_count()  # Number of available CPU cores
    pool = multiprocessing.Pool(processes=num_processes)
    
    # Define a partial function to pass additional arguments to the process_position function
    process_position_partial = partial(process_position, base_dir=base_dir, metadata=metadata)
    
    # Iterate over positions and keys in parallel
    for position, key in assay_layout.iterrows():
        if position == (3, 5):
            continue
            
        pool.apply_async(process_position_partial, args=(position,))
    
    # Close the multiprocessing pool and wait for all processes to finish
    pool.close()
    pool.join()


[INFO][2023/07/11 01:38:20 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/07/11 01:38:22 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
[INFO][2023/07/11 01:38:24 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 7).h5...
[INFO][2023/07/11 01:38:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 8).h5...
[INFO][2023/07/11 01:38:29 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 9).h5...
[INFO][2023/07/11 01:38:29 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/07/11 01:38:29 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5


Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/07/11 01:38:31 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(4, 4).h5...
[INFO][2023/07/11 01:38:33 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/07/11 01:38:33 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5


Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/07/11 01:38:34 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(4, 5).h5...
[INFO][2023/07/11 01:38:35 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/07/11 01:38:35 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 7).h5


...

Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/07/11 01:39:29 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/07/11 01:39:29 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 8).h5


Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/07/11 01:39:32 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/07/11 01:39:32 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 9).h5


Iterating over frames:   0%|          | 0/75 [00:00<?, ?it/s]